In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
import openpyxl  # or import xlsxwriter
import logging
import json
from pandas import Timestamp
import calendar
from collections import deque

# Setup and Configuration
EDOOFA_API_ENDPOINT = "https://edoofa-portal.bubbleapps.io/version-test/api/1.1/obj"  # Replace with your API endpoint
UMS_API_ENDPOINT = "https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj"  # UMS API endpoint
EDOOFA_API_TOKEN = "2cde31d8f48919a2db1467cc06a56132"  # Edoofa app API token
UMS_API_TOKEN = "8caeed4a8be2be66b70a76bcd486d4bb"  # UMS app API token

def get_headers(endpoint):
    if "edoofa-portal" in endpoint:
        return {"Authorization": f"Bearer {EDOOFA_API_TOKEN}", "Content-Type": "application/json"}
    elif "edoofa-ums" in endpoint:
        return {"Authorization": f"Bearer {UMS_API_TOKEN}", "Content-Type": "application/json"}
    else:
        raise ValueError("Unknown endpoint")

def fetch_all_attendance_data(start_date, end_date):
    all_data = []
    cursor = 0
    limit = 100
    total_records = None
    endpoint = f"{EDOOFA_API_ENDPOINT}/engagement"
    headers = get_headers(endpoint)

    while total_records is None or cursor < total_records:
        try:
            constraints = [
                {'key': 'engagement-date', 'constraint_type': 'greater than', 'value': start_date},
                {'key': 'engagement-date', 'constraint_type': 'less than', 'value': end_date}
            ]
            params = {
                'constraints': json.dumps(constraints),
                'cursor': cursor,
                'limit': limit
            }
            response = requests.get(endpoint, headers=headers, params=params)
            response.raise_for_status()

            data = response.json()
            records = data['response']['results']
            all_data.extend(records)
            
            if total_records is None:
                total_records = data['response']['remaining'] + len(records)
            cursor += len(records)

            logging.info(f"Fetched {len(records)} records. Total fetched: {cursor}.")

        except requests.exceptions.HTTPError as err:
            logging.error(f"HTTP error occurred: {err}")
            break
        except Exception as err:
            logging.error(f"Error occurred: {err}")
            break

    return all_data

def fetch_student_data():
    all_student_data = []
    cursor = 0
    limit = 100
    total_records = None
    endpoint = f"{EDOOFA_API_ENDPOINT}/student"
    headers = get_headers(endpoint)

    while total_records is None or cursor < total_records:
        try:
            params = {'cursor': cursor, 'limit': limit}
            response = requests.get(endpoint, headers=headers, params=params)
            response.raise_for_status()

            data = response.json()
            records = data['response']['results']
            all_student_data.extend(records)
            
            if total_records is None:
                total_records = data['response']['remaining'] + len(records)
            cursor += len(records)

        except requests.exceptions.HTTPError as err:
            logging.error(f"HTTP error occurred: {err}")
            break
        except Exception as err:
            logging.error(f"Error occurred: {err}")
            break
    
    if all_student_data: 
        print("Sample student data:", all_student_data[0])

    return all_student_data

def map_student_info(student_id, student_data):
    for student in student_data:
        # Using '_id' as the key to match the student ID
        if student['_id'] == student_id:
            ewyl_group_name = student.get('EWYL-group-name', 'Unknown')
            kam_group_name = student.get('KAM-group-name', 'Unknown')
            return ewyl_group_name, kam_group_name
    return 'Unknown', 'Unknown'

# Data Preprocessing
def process_attendance_data(data):
    df = pd.DataFrame(data)
    return df

# Data Preprocessing
def process_and_export_data(df, student_data):
    # Map student ID to EWYL and KAM group names
    mapped_values = df['student'].apply(lambda x: map_student_info(x, student_data))
    df['ewyl-group-name'], df['admissions-group-name'] = zip(*mapped_values)

    # Filter for only 'IE Call' or 'IE Chat' in 'engagement-type'
    df = df[df['engagement-type'].isin(['IE Call', 'IE Chat', 'Activity', 'Lesson'])]

    # Select specific columns and rename them
    columns_to_select = {
        'engagement-date': 'date',
        'engagement-type': 'attendance-type',
        'daily-attendance': 'present',
        'ewyl-group-name': 'ewyl-group-name',
        'admissions-group-name': 'admissions-group-name'
    }
    
    # Create a new DataFrame with the selected columns
    processed_df = df[list(columns_to_select.keys())]
    processed_df = processed_df.rename(columns=columns_to_select)

    return processed_df

def calculate_average_till_last_12_months(grouped_data):
    # Initialize the column with 0.0
    grouped_data['avg-att-percent-till-last-month'] = 0.0  

    for name, group in grouped_data.groupby(['ewyl-group-name', 'admissions-group-name']):
        # Keep only the last 12 attendance percentages
        last_12_months_attendance = deque(maxlen=12)
        for i, row in group.iterrows():
            # Append current month's attendance to the list
            last_12_months_attendance.append(row['attendance-percentage'])
            
            # If there are attendance records for at least 12 months, calculate the average
            if len(last_12_months_attendance) == 12:
                avg_last_12_months = sum(last_12_months_attendance) / 12
                grouped_data.at[i, 'avg-att-percent-till-last-month'] = avg_last_12_months

    return grouped_data

def create_aggregate_attendance_sheet(processed_data):
    # Convert 'date' to datetime and extract month and year
    processed_data['date'] = pd.to_datetime(processed_data['date'])
    processed_data['month'] = processed_data['date'].dt.strftime("%B")  # Use '%B' to get the full month name
    processed_data['year'] = processed_data['date'].dt.year

    # Calculate attendance count for each group by month and year
    attendance_count = processed_data.groupby(['ewyl-group-name', 'admissions-group-name', 'month', 'year', 'present']).size().unstack(fill_value=0)

    # Calculate attendance percentage
    total_classes = attendance_count.sum(axis=1)
    total_present = attendance_count.get(True, 0)
    attendance_count['attendance-percentage'] = (total_present / total_classes) * 100

    # Reset index for easy manipulation
    aggregate_data = attendance_count.reset_index()

    # Calculate the average attendance percentage until the last 12 months for each group
    for index, row in aggregate_data.iterrows():
        group = row['ewyl-group-name']
        year = row['year']
        month_index = list(calendar.month_name).index(row['month'])

        # Create a list to store the last 12 months' attendance percentages
        last_12_months_attendance = []

        # Loop over the last 12 months
        for i in range(1, 13):
            check_month_index = (month_index - i) % 12 or 12
            check_month_name = calendar.month_name[check_month_index]
            check_year = year if month_index - i > 0 else year - 1

            # Find the matching month and year in the data
            matching_data = aggregate_data[
                (aggregate_data['ewyl-group-name'] == group) & 
                (aggregate_data['year'] == check_year) & 
                (aggregate_data['month'] == check_month_name)
            ]

            # Append the average attendance percentage if data is available
            if not matching_data.empty:
                last_12_months_attendance.append(matching_data['attendance-percentage'].values[0])

        # Calculate the average of the last 12 months
        if last_12_months_attendance:
            avg_last_12_months = sum(last_12_months_attendance) / len(last_12_months_attendance)
        else:
            avg_last_12_months = 0

        # Update the current row with the calculated last 12 months average attendance
        aggregate_data.at[index, 'avg-att-percent-till-last-month'] = avg_last_12_months

    # Select and rename columns to match the Bubble structure
    aggregate_data = aggregate_data[['ewyl-group-name', 'admissions-group-name', 'attendance-percentage', 'avg-att-percent-till-last-month', 'month', 'year']]
    aggregate_data.columns = ['ewyl-group-name', 'admissions-group-name', 'attendance-percentage', 'avg-att-percent-till-last-month', 'month', 'year']

    return aggregate_data

def prepare_record_for_upload(row, columns_to_include):
    # Convert the DataFrame row to a dictionary
    record = row.to_dict()

    # Include only the desired columns
    filtered_record = {key: value for key, value in record.items() if key in columns_to_include}

    # Convert Timestamp objects to strings (ISO format)
    for key, value in filtered_record.items():
        if isinstance(value, pd.Timestamp):
            filtered_record[key] = value.isoformat()

    return filtered_record


def upload_to_bubble(data, endpoint, columns_to_include):
    headers = get_headers(endpoint)  # Get the correct headers for the endpoint

    for index, row in data.iterrows():
        record = prepare_record_for_upload(row, columns_to_include)  # Pass columns_to_include to prepare_record_for_upload

        try:
            response = requests.post(endpoint, headers=headers, json=record)
            if response.status_code == 200:
                print(f"Record {index} uploaded successfully")
            else:
                print(f"Error uploading record {index}: {response.text}")
        except Exception as e:
            print(f"Exception occurred while uploading record {index}: {e}")


def save_to_json_file(data, filepath):
    # Convert DataFrame to a list of dictionaries
    records = data.to_dict(orient='records')
    # Serialize to a JSON formatted string
    json_data = json.dumps(records, indent=4, default=str)
    
    # Write JSON data to file
    with open(filepath, 'w') as file:
        file.write(json_data)
    print(f"Data exported to JSON file at {filepath}")


# Main Function
def main():
    # Define your desired date range
    start_date = '2023-01-01'
    end_date = '2024-01-18'

    # Fetch attendance and student data
    original_data = fetch_all_attendance_data(start_date, end_date)
    student_data = fetch_student_data()

    if original_data:
        original_df = pd.DataFrame(original_data)

        # Process data with EWYL and KAM group name mapping
        processed_data = process_and_export_data(original_df, student_data)

        excel_file_path = 'C:\\Users\\ayush\\Documents\\UMS\\attendance_data5.xlsx'

        try:
            with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
                # Export processed data to Excel
                original_df.to_excel(writer, sheet_name='Original Data', index=False)
                processed_data.to_excel(writer, sheet_name='Processed Data', index=False)
            print("Data exported to Excel file.")
        except Exception as e:
            print(f"An error occurred: {e}")
        
        # Define columns to include in JSON for processed data
        columns_to_include_processed = ['date', 'attendance-type', 'present', 'ewyl-group-name', 'admissions-group-name']

        # Create aggregate attendance data
        aggregate_data = create_aggregate_attendance_sheet(processed_data)

        # Export aggregate attendance data to Excel
        with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a') as writer:
            aggregate_data.to_excel(writer, sheet_name='Aggregate Attendance', index=False)

        json_file_path_processed = excel_file_path.replace('.xlsx', '_processed.json')
        save_to_json_file(processed_data[columns_to_include_processed], json_file_path_processed)
        
        # Define columns to include in JSON for aggregated data
        columns_to_include_aggregated = ['ewyl-group-name', 'admissions-group-name', 'attendance-percentage', 'avg-att-percent-till-last-month', 'month', 'year']
        
        json_file_path_aggregated = excel_file_path.replace('.xlsx', '_aggregated.json')
        save_to_json_file(aggregate_data[columns_to_include_aggregated], json_file_path_aggregated)

        # Define API endpoints for uploading data to Bubble
        processed_data_upload_endpoint = UMS_API_ENDPOINT + "/Attendance"
        aggregated_data_upload_endpoint = UMS_API_ENDPOINT + "/Attendance Summary"

        # Upload Processed Data to Bubble
        upload_to_bubble(processed_data[columns_to_include_processed], processed_data_upload_endpoint, columns_to_include_processed)

        # Upload Aggregated Data to Bubble
        upload_to_bubble(aggregate_data[columns_to_include_aggregated], aggregated_data_upload_endpoint, columns_to_include_aggregated)

if __name__ == "__main__":
    main()
